In [1]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nlp = spacy.load('pt_core_news_sm')

In [3]:
df = pd.read_json("../data/raw/palestras.json")
df.reset_index(inplace=True)

In [4]:
df.head()

,index,title,description,trilha,ano,cidade,trilha_url
0,1000,KEAS - Kit de Adaptação Esportiva para Surdos,O projeto foi ocorrido do meu TCC da Universid...,trilha-internet-das-coisas,2017,florianopolis,http://www.thedevelopersconference.com.br/tdc/...
1,10000,Data Driven Organizations: como este conceito ...,"Que conceito é esse analisado pela Forbes, pel...",trilha-data-science,2017,portoalegre,http://www.thedevelopersconference.com.br/tdc/...
2,10001,"Aplicações, técnicas e ferramentas para descob...",A área de Descoberta de Conhecimento em Textos...,trilha-data-science,2017,portoalegre,http://www.thedevelopersconference.com.br/tdc/...
3,10002,Mineração de processos em ação - da teoria a r...,Mineração de processos é uma disciplina relati...,trilha-data-science,2017,portoalegre,http://www.thedevelopersconference.com.br/tdc/...
4,10003,Aplicando uma Camada de Ciência no Marketing D...,A Ciência de Dados vem conquistando espaço em ...,trilha-data-science,2017,portoalegre,http://www.thedevelopersconference.com.br/tdc/...


In [5]:
df.shape

(3209, 7)

### 1. Juntar título e descrição

In [6]:
palestras = df[["title","description"]].to_dict(orient="records")
# string única'
palestras = [k["title"]+" "+k["description"] for k in palestras]

### 2. Extrair o lemma de cada palavra a remover stopwords

In [8]:
palestras_sem_stopwords = []
palestras_docs = list(nlp.pipe(palestras, disable=["parser","ner"]))

for palestra in palestras_docs:
    tokens = [t.lemma_ for t in palestra if (not t.is_stop and t.is_alpha and (not t.pos_ == "ADV"))]
    palestra = " ".join(tokens)
    palestras_sem_stopwords.append(palestra)

### 3. Criar matriz com o TfidfVectorizer

In [9]:
vect = TfidfVectorizer()
tfidf = vect.fit_transform(palestras_sem_stopwords)
tfidf

<3209x11010 sparse matrix of type '<class 'numpy.float64'>'
	with 91941 stored elements in Compressed Sparse Row format>

### 4. Computar similaridade dos cosenos

In [14]:
input_data = "O Pipeline do Scikit-learn é um objeto da biblioteca que facilita bastante o processamento de dados, sendo possível tanto treinar quando fazer predições utilizando a mesma Pipeline. Usar os modelos no Jupyter Notebook é uma das tarefas que realizamos durante as análises, mas o objetivo final de muitas soluções é integrar os modelos com alguma aplicação (normalmente aplicações web). Nesta palestra vamos treinar um modelo usando o scikit-learn e usar o Flask para construir uma API REST capaz de disponibilizar o modelo como um serviço web."
#input_data = "Testes de hardware geralmente são muito detalhados. Os testes são necessários para garantir que todos os componentes de um sistema estejam operando como deveriam e que o sistema esteja funcionando exatamente de acordo com os requisitos especificados. Nessa palestra, serão apresentados alguns dos principais processos a serem seguidos para testar um Hardware recém desenvolvido."

input_data = list(nlp.pipe([input_data], disable=["parser","ner"]))[0]
input_data = [t.lemma_ for t in input_data if (not t.is_stop and t.is_alpha)]
input_data = " ".join(input_data)

y = vect.transform([input_data])

cosine_similarities = pd.Series(cosine_similarity(tfidf, y).flatten())

results = []
for index,score in cosine_similarities.nlargest(int(5)).iteritems():
    palestra = {}
    palestra["trilha"] = df.iloc[index].trilha
    palestra["cidade"] = df.iloc[index].cidade
    palestra["ano"] = df.iloc[index].ano
    palestra["descricao"] = palestras[index]
    palestra["similaridade"] = score
    results.append(palestra)

In [26]:
from IPython.core.display import HTML

pd.set_option('display.max_colwidth', -1)
HTML(pd.DataFrame(results).to_html())

,ano,cidade,descricao,similaridade,trilha
0,2018,portoalegre,"Como criar um Android com cérebro de pássaro Nesta palestra descreveremos um jeito simples e rápido de embutir modelos de inteligência artificial em dispositivos Android, através da serialização de objetos Java. Iremos gerar um modelo de conhecimento a partir de técnicas clássicas de machine learning com o auxílio da API Weka, realizando o treino desse modelo em um computador e gerando predições diretamente no celular.",0.244188,trilha-inteligencia-artificial
1,2018,florianopolis,"Marvin - Inteligência artificial de A a Z Marvin é a plataforma de código aberto que está ajudando os times de ciência de dados do maior e-commerce da América Latina a produzir modelos capazes de realizar a precificação dinâmica de itens na web e detectar fraudes nas operações financeiras durante a Black Friday. O Marvin é composto de ferramentas, bibliotecas e um poderoso servidor de aplicação que expõe micro serviços capazes de realizar predições em milissegundos.",0.188860,trilha-machine-learning
2,2018,portoalegre,"Criando API REST com python flask e mongodb Uma introdução a api rest, utilizando a linguagem python com framework flask e banco de dados noSQL mongodb, apresentar os conceitos de api, motivação a usar a linguagem python e vantagens de utilizar banco noSQL.",0.183722,trilha-python
3,2018,saopaulo,Utilizando Serviços Cognitivos nos seus Apps! Saiba como utilizar todo o poder de bibliotecas como CoreML e TensorFlow para fazer com que seus aplicativos tirem proveito de modelos de aprendizado de máquina treinados para resolver problemas como reconhecimento de imagens. E o melhor: Offline!,0.183683,trilha-computacao-cognitiva
4,2016,florianopolis,"Como modelar, integrar e desenvolver aplicações com mútliplos bancos de dados NoSQL Com a evolução dos bancos de dados NoSQL tornou-se comum o desenvolvimento de aplicações cujos dados estão armazenados - e replicados - em múltiplos bancos de dados. Nesse cenário, modelar, integrar e desenvolver não é mais como era antigamente. Essa palestra discute arquiteturas, ferramentas e novos padrões de desenvolvimento que permitem explorar ao máximo as capacidades os diversos modelos NoSQL e facilitam sua integração com técnicas tradicionais do desenvolvimento de software.",0.178383,trilha-nosql
